In [ ]:
# Source Code in Python Jupyter Lab für ein elementares Fraud Detection Modell

# VORLESUNG 1:

# Importiere Library Pandas in Python

import pandas as pd
import os

# Lade das CSV File
df = pd.read_csv("onlinefraud.csv")

# Schreibe die Datenstruktur aus dem DataFrame auf dem Bildschirm
print(df)

# Schreibe spezifische Elemente und die Datentypen innerhalb von DataFrame
print(df.iloc[8])
print(df.dtypes)

# Schreibe die statistische Kennzahlen
df.describe()

# VORLESUNG 2:

# Selektiere nur die numerischen Werte
numeric_df = df.select_dtypes(include='number')
print(numeric_df)

# Berechne die Korrelationen
print(numeric_df.corr())

# Selektiere nur die nicht-numerischen Werte
non_numeric_df = df.select_dtypes(exclude='number')
print(non_numeric_df)

# VORLESUNG 3:

# Innerhalb der Library Sklearn - verwende einen Label Encoder 
import sklearn
from sklearn.preprocessing import LabelEncoder

# Encodiere alle nicht-numerischen Spalten
for col in df.select_dtypes(exclude='number').columns:

    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Schreibe die Korrelationen der enkodierten (nicht-numerischen) Werte
print(df)
print(df.corr())

# Lade die CSV Daten neu in den Arbeitsspeicher
df = pd.read_csv("onlinefraud.csv")
print(df)

# Projiziere die Daten auf den Unterraum der als Fraud deklarierten Einheiten
FindFraudStructures = df.loc[df["isFlaggedFraud"] == 1, ["nameDest", "isFlaggedFraud", "amount"]].copy()
print(FindFraudStructures)

# Zähle die Anzahl der als Fraud deklarierten Einheiten
FindFraudStructures.count()

# VORLESUNG 4:

# Lege eine Klassifizierung für die als Fraud deklarierten Transaktionen fest
FindFraudStructures["ClassifyFraud"] = pd.cut(
    df["amount"],
    bins=[0, 100.0, 100000.0, 100000000.0],         # Grenzen der Intervalle
    labels=["no fraud", "fraud", "extreme fraud"]  # Namen für die Gruppen
)

# Gebe die Datenstruktur aus
print(FindFraudStructures)

# Speichere die als Fraud deklarierten Transaktionen in einem TXT File
FindFraudStructures.to_csv("FraudPatterns.txt", sep="\t", index=False, encoding="utf-8")

# VORLESUNG 5:

# Erzeuge einen beispielhaften Datensatz (Generative Daten) aus einem generativen Modell zur Datenerzeugung
import pandas as pd
import numpy as np

## Reproduzierbarkeit
np.random.seed(42)

## Anzahl der Samples
n_samples = 1000

## Zeitliche Dimension
steps = np.random.randint(1, 744, size=n_samples)  # ähnlich wie 1–743 im Original

## Transaktionstypen
types = np.random.choice(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
                         size=n_samples)

## Beträge (positiv, lognormal verteilt -> viele kleine, wenige große)
amounts = np.random.lognormal(mean=10, sigma=1, size=n_samples) / 100

## Sender-IDs (z. B. Kundenkonten)
nameOrig = ["C" + str(np.random.randint(100000000, 999999999)) for _ in range(n_samples)]

## Ursprüngliches Guthaben (immer >= amount)
oldbalanceOrg = amounts + np.random.uniform(0, 1e6, size=n_samples)

## Neues Guthaben
newbalanceOrg = oldbalanceOrg - amounts
newbalanceOrg = np.clip(newbalanceOrg, 0, None)  # keine negativen Salden

## Empfänger-IDs (C... für Kunden, M... für Händler)
nameDest = []
for _ in range(n_samples):
    if np.random.rand() < 0.7:  # 70% gehen an Händler
        nameDest.append("M" + str(np.random.randint(100000000, 999999999)))
    else:
        nameDest.append("C" + str(np.random.randint(100000000, 999999999)))

## Ziel-Salden (vereinfachtes Modell)
oldbalanceDest = np.random.uniform(0, 1e6, size=n_samples)
newbalanceDest = oldbalanceDest + amounts

## Fraud-Indikator (z. B. 1% Fraud)
isFraud = np.random.choice([0, 1], size=n_samples, p=[0.95, 0.05])

## Flagged Fraud (z. B. wenn Betrag > 200.000)
isFlaggedFraud = (amounts > 200).astype(int)

## DataFrame bauen
df = pd.DataFrame({
    'step': steps,
    'type': types,
    'amount': np.round(amounts, 2),
    'nameOrig': nameOrig,
    'oldbalanceOrg': np.round(oldbalanceOrg, 2),
    'newbalanceOrig': np.round(newbalanceOrg, 2),
    'nameDest': nameDest,
    'oldbalanceDest': np.round(oldbalanceDest, 2),
    'newbalanceDest': np.round(newbalanceDest, 2),
    'isFraud': isFraud,
    'isFlaggedFraud': isFlaggedFraud
})

## CSV speichern
df.to_csv("Synthetic_Transactions.csv", index=False)

print("Synthetic dataset saved as 'Synthetic_Transactions.csv'")

# VORLESUNG 7:

# Neues CSV File laden
df = pd.read_csv("Synthetic_Transactions.csv")
value_count = 0.0

# Durch jede Zeile von FindFraudStructures iterieren
for i in range(len(FindFraudStructures)):

    value = FindFraudStructures["nameDest"].iloc[i]

    if value in df["nameDest"].values:
    
        print(value)
    
    if value in df["nameDest"].values:
    
        value_count = value_count + 1

print(int(value_count))
print(len(df["nameDest"]))

# VORLESUNG 8:

# Im folgenden wird ein generatives Modell definiert

## Importieren die richtigen Libraries

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Lade CSV Daten
df = pd.read_csv("Synthetic_Transactions.csv")

# Kategorische Variablen encoden
label_encoders = {}

for col in ["type", "nameOrig", "nameDest"]:

    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Features und Ziele definieren
input_data = df.drop(["isFraud", "isFlaggedFraud"], axis=1)
output_data = df["isFraud"]

# Daten normalisieren (SVM mag skalierte Daten)
scaler = StandardScaler()
input_data_scaled = scaler.fit_transform(input_data)

# Train/Test Splitting festlegen
input_data_train, input_data_test, output_data_train, output_data_test = train_test_split(
    input_data_scaled, output_data, test_size=0.3, random_state=42, stratify=output_data
)

# SVM Modell definieren (Support Vector Maschine)
svm = SVC(kernel="rbf", class_weight="balanced")  # "balanced" falls Fraud selten ist
svm.fit(input_data_train, output_data_train)

# Vorhersagen generieren 
output_data_pred = svm.predict(input_data_test)

# Ergebnisse in einer Konfusionsmatrix speichern
print("Confusion Matrix:\n", confusion_matrix(output_data_test, output_data_pred))
print("\nClassification Report:\n", classification_report(output_data_test, output_data_pred))

# Spezifische Vorhersage generieren
# Eingabedaten (als DataFrame mit den gleichen Spalten wie die Eingabedaten, welche in der Variablen input_data gespeichert sind)

new_transaction = pd.DataFrame([{
    "step": 118,
    "type": "PAYMENT",
    "amount": 180.00,
    "nameOrig": "C613567293",
    "oldbalanceOrg": 595905.09,
    "newbalanceOrig": 595905.09,
    "nameDest": "C967937996",
    "oldbalanceDest": 105402.96,
    "newbalanceDest": 115228.23
}])

# Gleiche LabelEncoder verwenden wie beim Training
for col, le in label_encoders.items():
    new_transaction[col] = le.transform(new_transaction[col])

# Skalieren wie beim Training
new_transaction_scaled = scaler.transform(new_transaction)

# Vorhersage generieren
output_data_pred = svm.predict(new_transaction_scaled)

# Vorhersagewert schreiben
print("Vorhersage (0 = kein Fraud, 1 = Fraud):", output_data_pred[0])


